<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Data input and text removal testing testing

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers

import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filepath = '/content/drive/MyDrive/Text-ML/full_sentiment_dataset.csv' #'data.csv'
df= pd.read_csv(filepath)
df1=df.drop(['no','paper'],axis=1)
df1.head()

,cited_paper,label,text,context_a,context_b
0,A00-2024,0,We analyzed a set of articles and identified s...,NaN,NaN
1,A00-2024,0,Table 3: Example compressions Compression AvgL...,NaN,NaN
2,A00-2024,0,5.3 Related works and discussion Our two-step ...,NaN,NaN
3,A00-2024,0,(1999) proposed a summarization system based o...,NaN,NaN
4,A00-2024,0,We found that the deletion of lead parts did n...,NaN,NaN


In [5]:
df1['label'].value_counts()

 0    7627
 1     829
-1     280
Name: label, dtype: int64

In [6]:
context=df1['text']
# pp.pprint(context[0])
# pp.pprint(context[1])
# pp.pprint(context[2])
# pp.pprint(context[3])
# pp.pprint(context[4])

re1= "\(((([A-Za-z]+ *)+(, \d+))+(; )*)+\)" # matches author and author, year
re_year=",? \(?\d{4}\)?" # match , {4 digits} which may be wrapped in () 
re_et= "(\w+ *)et al\. ?"+re_year # matches author et al. , year
re_2a= "(\w+ *)and ((\w+ *))?"+re_year # matches author and author, year
re_sep="((; )|( and ))*"# match the '; ' gap or ' and ' gap
re_apa ="\(?("+"(\(?"+re_2a+"|"+re_et+"\)?)"+re_sep+")+"
p1= re.compile(re_apa)
a=p1.search(context[0])
print(a)
a=p1.search(context[2])
print(a)
#p1= re.compile(re_2a)
a=p1.search(context[4])
print(a)
print(a.groups())

<re.Match object; span=(437, 485), match='(Jing and McKeown, 1999; Jing and McKeown, 2000)'>
<re.Match object; span=(109, 158), match='(Mani et al. , 1999) and (Jing and McKeown, 2000)>
<re.Match object; span=(101, 124), match='Jing and McKeown (2000)'>
('Jing and McKeown (2000)', 'Jing and McKeown (2000)', 'Jing ', 'McKeown', 'McKeown', None, None, None, None)


In [7]:
def remove_citation(text,regex=re_apa):
  text1=text
  rem_len=0
  pattern= re.compile(regex)
  while True:
    matches=pattern.search(text1)
    # print(matches)
    if matches == None:
      break

    spn=matches.span()
    text1=text1[0:spn[0]]+text1[spn[1]:-1]
    cit_len=spn[1]-spn[0]
    rem_len+=cit_len
  
  percent_removed=rem_len/len(text)
  return text1,percent_removed 

print(context[5])
remove_citation(context[5],regex=re_apa)

Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences (Lin and Hovy, 2002; Schiffman et al., 2002; Erkan and Radev, 2004; Mihalcea and Tarau, 2004; Daume III and Marcu, 2006) but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification (Jing and McKeown, 2000; Knight and Marcu, 2000; Barzilay and McKeown, 2005).


('Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences Daume  but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification ',
 0.4209354120267261)

In [8]:
output=df1['text'].apply(lambda x: remove_citation(text=x)) #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
df_o = pd.DataFrame(list(output), columns =['clean','p_rem'])
#df_o.head()

df1['text_clean']=df_o['clean']
df1['text_clean_len']=df_o['clean'].apply(len)
df1['p_rem']=df_o['p_rem']
df1.head()

,cited_paper,label,text,context_a,context_b,text_clean,text_clean_len,p_rem
0,A00-2024,0,We analyzed a set of articles and identified s...,NaN,NaN,We analyzed a set of articles and identified s...,437,0.098765
1,A00-2024,0,Table 3: Example compressions Compression AvgL...,NaN,NaN,Table 3: Example compressions Compression AvgL...,325,0.065903
2,A00-2024,0,5.3 Related works and discussion Our two-step ...,NaN,NaN,5.3 Related works and discussion Our two-step ...,109,0.308176
3,A00-2024,0,(1999) proposed a summarization system based o...,NaN,NaN,(1999) proposed a summarization system based o...,344,0.062500
4,A00-2024,0,We found that the deletion of lead parts did n...,NaN,NaN,We found that the deletion of lead parts did n...,101,0.184000


# Remove under and over sized samples
large samples appear to be poorly written

In [9]:
def getMidLen(data,label,labelKey='label',lenKey='text_clean_len',lowMod=1,highMod=1):
  df1 =data.loc[data[labelKey] == label]
  neu_mean=np.mean(list(df1[lenKey]))
  neu_std=np.std(list(df1[lenKey]))
  df1_no_high = df1.loc[df1[lenKey] < highMod*(neu_mean +neu_std)]
  # print(neu_mean)
  # print(neu_std)

  while neu_std > neu_mean:
    neu_mean=np.mean(list(df1_no_high['text_clean_len']))
    neu_std=np.std(list(df1_no_high['text_clean_len']))
    # print(neu_mean)
    # print(neu_std)
    df1_no_high = df1.loc[df1['text_clean_len'] < highMod*(neu_mean +neu_std)]

  df1_mid = df1_no_high.loc[df1_no_high['text_clean_len'] > lowMod*(neu_mean -neu_std)]

  return df1_mid

df2 = df1.loc[df1['p_rem'] < .5] #keep sampels with less than half of it are citation

df_neu=getMidLen(df2,0,lowMod=2)
df_pos=getMidLen(df2,1,lowMod=1,highMod=2)
df_neg=getMidLen(df2,-1,lowMod=1,highMod=2)
df3= pd.concat([df_neg,df_neu,df_pos])
df3['label'].value_counts()

 0    2745
 1     748
-1     250
Name: label, dtype: int64

In [10]:
def catagorize(data,labelKey='label'):
  rows=len(data.index)
  onehots=np.zeros((rows,3),dtype=int)
  for i,lab in enumerate(data[labelKey]):
    onehots[i][lab]=1
  return onehots

hots=catagorize(df3)
df3['label_onehot']=list(hots)
df3['label_index']=df3['label']+1

In [11]:
freq= np.array(list(df3['label_index'].value_counts(normalize=True,sort=False)))
print(freq)
class_ratio= 1/freq
class_ratio

[0.06679134 0.73336896 0.1998397 ]


array([14.972     ,  1.36357013,  5.0040107 ])

Trying a new model,

Rather https://www.tensorflow.org/text/tutorials/classify_text_with_bert

In [12]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"
# For adamW
!pip install -q tf-models-official==2.7.0 
# focal loss implmnetion for tf
!pip install focal-loss

     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 462 kB 61.1 MB/s 
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 90 kB 7.5 MB/s 
     |████████████████████████████████| 99 kB 8.4 MB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 
     |████████████████████████████████| 352 kB 57.3 MB/s 
     |████████████████████████████████| 47.8 MB 1.2 MB/s 
     |████████████████████████████████| 237 kB 62.2 MB/s 
     |████████████████████████████████| 1.2 MB 32.6 MB/s 


In [13]:
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from focal_loss import SparseCategoricalFocalLoss

# X_train, X_test, y_train, y_test = train_test_split(list(df3['text_clean']), list(df3['label_onehot']), test_size=0.2, random_state=42)
# X_train= [[s] for s in X_train]
# X_test= [[s] for s in X_test]

X_train, X_test, y_train, y_test = train_test_split(list(df3['text_clean']), list(df3['label_index']), test_size=0.2, random_state=42)
X_train= [[s] for s in X_train]
X_test= [[s] for s in X_test]
y_train=[[s] for s in list(y_train)]
y_test=[[s] for s in list(y_test)]

In [14]:
#@title Choose a BERT model to fine-tune (Taken from tutorial)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [15]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [17]:
text_test = X_train[1]
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2735 3240 1006 2294 1007 5577 1037 4118 1006 2397 3372]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [18]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [19]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.99165255 -0.9433876  -0.32921582 -0.06821639 -0.46427464  0.79109323
  0.98792756 -0.09634462 -0.56172603 -0.26840577 -0.32873237 -0.97133565]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.5579294   1.0139146   0.1347054  ... -0.15144181  0.45422712
   0.93827504]
 [-1.4914836   0.20700586  0.9332853  ...  1.2075826  -0.5134389
  -0.32814234]
 [-1.0656238  -0.66518766 -0.4961959  ...  0.2893991   1.3608509
  -0.4220311 ]
 ...
 [-0.85306126  0.16896583 -0.8424922  ... -0.14183459 -1.2360741
   1.124439  ]
 [-0.9763724  -0.69449544 -0.82873714 ... -1.0230062  -0.30265796
   0.32449037]
 [-0.68395424  0.9129317  -0.54485846 ... -0.48854524  0.6978089
   0.11847503]]


In [20]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [21]:
classifier_model = build_classifier_model()

In [22]:
bert_raw_result = classifier_model(tf.constant(text_test))
print(bert_raw_result)

l =  SparseCategoricalFocalLoss(gamma=2,class_weight=class_ratio)
test =tf.convert_to_tensor([1.0])
l(test,bert_raw_result)

tf.Tensor([[0.59094065 0.21518023 0.1938791 ]], shape=(1, 3), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=1.2902905>

In [23]:
epochs = 5
steps_per_epoch = 200 #tf.data.experimental.cardinality(X_train).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 2e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

# def auc_wrapper(y_true,y_pred):
#   print(y_true,y_pred)

#   y_true=tf.reshape(y_true,[1])
#   print(y_true)
#   y_true= tf.cast(y_true, tf.int32)
#   print(y_true)
#   y_true=tf.one_hot(y_true,depth=3)
#   print(y_true)
#   return tf.keras.metrics.AUC(multi_label=True)(y_true,y_pred)


loss =  SparseCategoricalFocalLoss(gamma=2,class_weight=class_ratio) #tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]#, auc_wrapper]#, tf.keras.metrics.AUC(multi_label=True)]


classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="citation_BERT_{epoch}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_sparse_categorical_accuracy",
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard('./logs', update_freq=1)
]


In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=X_train,y=y_train, validation_data=(X_test,y_test),epochs=epochs,callbacks= callbacks, verbose=True)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
94/94 [==============================] - ETA: 0s - loss: 0.1550 - sparse_categorical_accuracy: 0.9405
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.77704, saving model to citation_BERT_1


INFO:tensorflow:Assets written to: citation_BERT_1/assets


INFO:tensorflow:Assets written to: citation_BERT_1/assets


94/94 [==============================] - 764s 8s/step - loss: 0.1550 - sparse_categorical_accuracy: 0.9405 - val_loss: 2.5789 - val_sparse_categorical_accuracy: 0.7770
Epoch 2/5
94/94 [==============================] - ETA: 0s - loss: 0.1878 - sparse_categorical_accuracy: 0.9319
Epoch 2: val_sparse_categorical_accuracy improved from 0.77704 to 0.78505, saving model to citation_BERT_2


INFO:tensorflow:Assets written to: citation_BERT_2/assets


...

INFO:tensorflow:Assets written to: citation_BERT_3/assets


94/94 [==============================] - 791s 8s/step - loss: 0.1752 - sparse_categorical_accuracy: 0.9516 - val_loss: 2.3198 - val_sparse_categorical_accuracy: 0.7864
Epoch 4/5
94/94 [==============================] - ETA: 0s - loss: 0.0874 - sparse_categorical_accuracy: 0.9656
Epoch 4: val_sparse_categorical_accuracy improved from 0.78638 to 0.79039, saving model to citation_BERT_4


INFO:tensorflow:Assets written to: citation_BERT_4/assets


INFO:tensorflow:Assets written to: citation_BERT_4/assets


94/94 [==============================] - 759s 8s/step - loss: 0.0874 - sparse_categorical_accuracy: 0.9656 - val_loss: 2.4526 - val_sparse_categorical_accuracy: 0.7904
Epoch 5/5
94/94 [==============================] - ETA: 0s - loss: 0.1207 - sparse_categorical_accuracy: 0.9703
Epoch 5: val_sparse_categorical_accuracy did not improve from 0.79039
94/94 [==============================] - 790s 8s/step - loss: 0.1207 - sparse_categorical_accuracy: 0.9703 - val_loss: 2.9798 - val_sparse_categorical_accuracy: 0.7744


In [ ]:
! pip install TensorBoard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
#%tensorboard --logdir=logs

https://towardsdatascience.com/deconstructing-bert-part-2-visualizing-the-inner-workings-of-attention-60a16d86b5c1


In [54]:
classifier_model.layers

In [69]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 


In [70]:
from transformers import BertTokenizer, BertModel

In [66]:
preprocesser_model = keras.Model(inputs=classifier_model.input,
                                       outputs=classifier_model.get_layer('preprocessing').output)

encoder_model = keras.Model(inputs=classifier_model.input,
                                       outputs=classifier_model.get_layer('BERT_encoder').output)

text_test = X_train[1]
test_tensor=tf.convert_to_tensor(text_test)
print("input:",test_tensor)

p_out=preprocesser_model(test_tensor)
text_preprocessed = bert_preprocess_model(text_test)

stop_index=0
while(text_preprocessed["input_mask"][0][stop_index] == 1):
  stop_index+=1
print("stop id index:",stop_index)

input_id_list = text_preprocessed["input_word_ids"][:,0:stop_index]
print(input_id_list)

output = encoder_model(test_tensor)
res=tf.math.reduce_mean(output["sequence_output"],axis=-1)
#print(res.shape)
#print(res)
regulatized=tf.keras.activations.softmax(res)[:,0:stop_index]
#print(regulatized.shape)
print(regulatized)

input: tf.Tensor([b'Turney (2006) describes a method (Latent Relational Analysis) that extracts subsequence patterns for noun pairs from a large corpus, using query expansion to increase the recall of the search and feature selection and dimensionality reduction to reduce the complexity of the feature space.'], shape=(1,), dtype=string)
stop id index: 57
tf.Tensor(
[[  101  2735  3240  1006  2294  1007  5577  1037  4118  1006  2397  3372
  28771  4106  1007  2008 27059  4942  3366  4226  5897  7060  2005 15156
   7689  2013  1037  2312 13931  1010  2478 23032  4935  2000  3623  1996
   9131  1997  1996  3945  1998  3444  4989  1998  8789  3012  7312  2000
   5547  1996 11619  1997  1996  3444  2686  1012   102]], shape=(1, 57), dtype=int32)
tf.Tensor(
[[0.00778976 0.00779011 0.00780449 0.00779925 0.00778113 0.0077906
  0.00779897 0.00781448 0.00780328 0.00779048 0.0077724  0.00779511
  0.00779415 0.00779945 0.00780737 0.00780792 0.00779753 0.00778464
  0.00776503 0.00779814 0.00780669 

In [73]:
model_version = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_version) # ok so this and our other one encode tokens the Same!

In [72]:
tokens = tokenizer.convert_ids_to_tokens(text_preprocessed["input_word_ids"][0]) 
print(tokens)

['[CLS]', 'turn', '##ey', '(', '2006', ')', 'describes', 'a', 'method', '(', 'late', '##nt', 'relational', 'analysis', ')', 'that', 'extracts', 'sub', '##se', '##que', '##nce', 'patterns', 'for', 'noun', 'pairs', 'from', 'a', 'large', 'corpus', ',', 'using', 'query', 'expansion', 'to', 'increase', 'the', 'recall', 'of', 'the', 'search', 'and', 'feature', 'selection', 'and', 'dimensional', '##ity', 'reduction', 'to', 'reduce', 'the', 'complexity', 'of', 'the', 'feature', 'space', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'